<a href="https://colab.research.google.com/github/Newbie-Nyx/simple-img2img/blob/main/img2img_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#注意！このノートはhuggingfaceのトークンを持っているという前提で書いています。
#分かりにくいところだらけですが試作品に等しいものなのでご容赦ください。

In [ ]:
#@title 手順1/3　トークンをコピペした後に実行ボタンをクリック
!pip install transformers scipy ftfy
!pip install git+https://github.com/huggingface/diffusers.git
#@title 手順2/4　トークンをコピぺして実行ボタンをクリック
#@markdown 右のURLに飛んでトークンを取得し、下に貼り付けてください。 (https://huggingface.co/settings/tokens)
YOUR_TOKEN="トークン"#@param {type:"string"}
#@markdown　一度張り付けた後にセーブすれば、２回目以降はURL先に飛ばなくても残っていると思います。
import torch
from diffusers import StableDiffusionImg2ImgPipeline

# パイプラインの準備
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=YOUR_TOKEN
).to("cuda")

In [ ]:
#@title 手順2/3　実行ボタンをクリックし、画像をアップロード
import os
from google.colab import files

uploaded = files.upload()
uploaded = list(uploaded.keys())
assert len(uploaded) == 1


In [ ]:
#@title 手順3/3　実行ボタンをクリックし、img2imgを実行
from google.colab import files        
from torch.random import seed
from torch import autocast 
from pathlib import Path
from PIL import Image
import random
import shutil
import os
#-----------------------------------------------------------------------------------------
try:
    shutil.rmtree('/A')               #フォルダAを消す
    os.remove('/AIart.zip')           #zipファイルを消す
except FileNotFoundError:
    pass
#-----------------------------------------------------------------------------------------
path = '/A'                           #フォルダAを作成
os.makedirs(path, exist_ok=True)
#-----------------------------------------------------------------------------------------
#@markdown i : 出力したい枚数
i = 1 #@param {type:"slider", min:1, max:50, step:1}

for I in range(i):

# 画像生成
  prompt = "style of Gogh"#@param {type:"string"}
  init_image = Image.open(uploaded[0]).convert("RGB")
  width  = 512 #@param {type:"slider", min:512, max:960, step:64}
  height = 640 #@param {type:"slider", min:512, max:960, step:64}
  init_image = init_image.resize((width, height))
  #@markdown strengh : 0.00(ほとんど変化なし)~1.00(全然違う画像)
  Strength = 0.667 #@param {type:"slider", min:0, max:1, step:0.001} 
  #@markdown step : step数が高い程書き込みが精緻になります。（正直50前後が安定です。）
  Step  = 45 #@param {type:"slider", min:1, max:100, step:1}

  with autocast("cuda"):
      images = pipe(
          prompt=prompt, 
          init_image=init_image, 
          strength=Strength, 
          guidance_scale=7.5,
          num_inference_steps=Step,
          generator=None,
          output_type="pil",          
          )["sample"]
  images[0].save(f"/A/{I + 1}.png")
#-----------------------------------------------------------------------------------------
if i == 1:
  files.download(f"/A/{I + 1}.png")
else:
  !zip -r /AIart.zip /A                   # フォルダAを zip 圧縮する

  from google.colab import files          # 圧縮したzipファイルをダウンロードする
  files.download("/AIart.zip")
#-----------------------------------------------------------------------------------------